# Dependências

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Configurações

In [ ]:
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = (8,6)
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['font.size'] = 15

# Utils

In [ ]:
voc = ["EU", "VOCÊ", "É", "SOU", "MUITO", "LEGAL", "DEMAIS", "INCRÍVEL"]

def encode(text):
  a_text = text.split(" ")
  r = []
  for i in a_text:
    if i in voc:
      r.append(voc.index(i))
  return np.array(r)

def probs(word):
  r = np.zeros(len(voc))
  i = voc.index(word)

  if i > -1:
    r[i] = 1.0

  return r

def decode(a_text):
  return voc[np.argmax(a_text)]

# Dataset

In [ ]:
inputs = np.array([
    encode("EU SOU MUITO"),
    encode("MUITO LEGAL EU"),
    encode("EU SOU LEGAL"),
    encode("VOCÊ É MUITO"),
    encode("VOCÊ É LEGAL")
    ])

targets = np.array([
    probs("LEGAL"),
    probs("SOU"),
    probs("DEMAIS"),
    probs("LEGAL"),
    probs("DEMAIS")
    ])

print(f"Inputs: \n{inputs}")
print(f"Targets: \n{targets}")

# Arquitetura (INCORRETA)

In [ ]:
class SimpleRNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Inicialização dos pesos e biases
        self.weights_input_hidden = np.random.uniform(size=(input_size, hidden_size))
        self.weights_hidden_hidden = np.random.uniform(size=(hidden_size, hidden_size))
        self.biases_hidden = np.zeros(hidden_size)

        self.weights_hidden_output = np.random.uniform(size=(hidden_size, output_size))
        self.biases_output = np.zeros(output_size)

        # Estado oculto
        self.hidden_state = np.zeros(hidden_size)

    # def sigmoid(self, x):
    #     return 1 / (1 + np.exp(-x))

    # def sigmoid_derivative(self, x):
    #     return x * (1 - x)

    def tanh(self, x):
        return np.tanh(x)

    def tanh_derivative(self, x):
        return 1.0 - np.tanh(x)**2

    def softmax(self, x):
      exp_x = np.exp(x)
      sum_exp_x = np.sum(exp_x, axis=-1, keepdims=True)
      return exp_x / sum_exp_x


    def train(self, inputs, targets, epochs, learning_rate):
        loss_history = []

        for epoch in range(epochs):
            total_loss = 0

            for i in range(len(inputs)):
                # Forward pass
                input_layer = inputs[i]
                hidden_layer_input = np.dot(input_layer, self.weights_input_hidden) + np.dot(self.hidden_state, self.weights_hidden_hidden) + self.biases_hidden
                hidden_layer_output = self.tanh(hidden_layer_input)
                #hidden_layer_output = self.sigmoid(hidden_layer_input)


                output_layer_input = np.dot(hidden_layer_output, self.weights_hidden_output) + self.biases_output
                predicted_output = output_layer_input

                # Calcula o erro
                error = targets[i] - predicted_output
                total_loss += np.sum(error ** 2)

                # Backpropagation
                output_delta = error
                hidden_delta = output_delta.dot(self.weights_hidden_output.T) * self.tanh_derivative(hidden_layer_output)
                #hidden_delta = output_delta.dot(self.weights_hidden_output.T) * self.sigmoid_derivative(hidden_layer_output)


                # Atualiza os pesos e biases
                self.weights_hidden_output += hidden_layer_output.reshape(-1, 1) * output_delta * learning_rate
                self.biases_output += output_delta * learning_rate

                self.weights_input_hidden += input_layer.reshape(-1, 1) * hidden_delta * learning_rate
                self.weights_hidden_hidden += np.outer(self.hidden_state, hidden_delta) * learning_rate
                self.biases_hidden += hidden_delta * learning_rate

                # Atualiza o estado oculto
                self.hidden_state = hidden_layer_output

            # Calcula o erro médio quadrático médio para a época
            avg_loss = total_loss / len(inputs)
            loss_history.append(avg_loss)

            # Exibe o erro médio quadrático a cada época
            print(f"Época {epoch + 1}/{epochs}, Erro Médio Quadrático: {avg_loss}")

        return loss_history

    def predict(self, initial_sequence):
        current_input = np.array(initial_sequence)
        hidden_layer_input = np.dot(current_input, self.weights_input_hidden) + np.dot(self.hidden_state, self.weights_hidden_hidden) + self.biases_hidden
        hidden_layer_output = self.tanh(hidden_layer_input)
        #hidden_layer_output = self.sigmoid(hidden_layer_input)

        output_layer_input = np.dot(hidden_layer_output, self.weights_hidden_output) + self.biases_output
        predicted_output = output_layer_input

        # Atualiza o estado oculto
        self.hidden_state = hidden_layer_output

        return predicted_output

# Treinamento

In [ ]:
# Hiperparâmetros
epochs = 8000
learning_rate = 0.001

# Cria e treina a RRN
rnn = SimpleRNN(input_size=len(inputs[0]), hidden_size=10, output_size=len(targets[0]))
losses = rnn.train(inputs, targets, epochs, learning_rate)


In [ ]:
data_epochs = np.arange(0, epochs, 1)

plt.plot(data_epochs, losses, color="yellow", lw=5, label = 'MSE')

plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()

# Teste

In [ ]:
# Testa a previsão com uma nova sequência
#test_sequence = [1, 0, 1]

text = "EU SOU MUITO" #Valor conhecido
# text = "VOCÊ É INCRÍVEL" #valor desconhecido
# text = "SOU MUITO INCRÍVEL" #valor desconhecido meio provável
# text = "VOCÊ É EU" #valor desconhecido improvável
# text = "EU SOU É" #valor desconhecido improvável

test_sequence = encode(text)
predicted_output = rnn.predict(test_sequence)

print(f"Vocabulário: {voc}")
print()
print(f"Frases conhecidas:\n")

for i in range(len(inputs)):
    s = ""
    for j in inputs[i]:
      s += voc[j] +" "
    print(f"{s}{decode(targets[i])}")

print()

print(f"Sequência de entrada: {test_sequence}")
print(f"Sequência prevista: {predicted_output}")
print()

print(f"Texto de entrada: {text} ...")
print(f"Texto de saída: {text} {decode(predicted_output)}. ({np.max(predicted_output)})")


## Questões

1. Corriga o código para gerar a distribuição de probabilidades das palavras do vocabulário.
2. Execute os diversos casos de teste a acima e observe os resultados.
3. Modifique para função sigmoid e derivada da sigmoid o treinamento. Os resultados foram piores? Melhores?